# Predictions: Brain Tumor Classif. with PyTorch⚡Lightning

The goal of this challenge is to Predict the status of a genetic biomarker important for brain cancer treatment.

All the code is refered from public repository: https://github.com/Borda/kaggle_brain-tumor-3D
Any nice contribution is welcome!

In [ ]:
! cp -r /kaggle/input/brain-tumor-classif-submissions/package_freeze package_freeze

import glob, os, shutil
# finad all packages
pkgs = glob.glob("package_freeze/*.xyz")
# rename them back to correct name format
[shutil.move(p, p.replace(".xyz", ".tar.gz")) for p in pkgs]

! pip install -q "kaggle-brain3D" --no-index --find-link /kaggle/working/package_freeze

# ! pip install -q https://github.com/Borda/kaggle_brain-tumor-3D/archive/refs/heads/main.zip

! pip uninstall -q -y wandb
! pip list | grep torch

In [ ]:
! ls -l /kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification
! nvidia-smi
# ! mkdir /kaggle/temp

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import kaggle_brain3d
print(kaggle_brain3d.__version__)

## Prepare dataset

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PATH_DATASET = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification"
PATH_MODELS = "/kaggle/input/brain-tumor-classif-submissions"
# PATH_TEMP = "/kaggle/temp"
SCAN_TYPES = ("FLAIR", "T1w", "T1CE", "T2w")

df_sample = pd.read_csv(os.path.join(PATH_DATASET, "sample_submission.csv"))
display(df_sample.head())

### Lightning DataModule

It is constric to wrap all data-related peaces and define Pytoch dataloder for Training / Validation / Testing phase.

At the end we show a few sample images from the fost training batch.

In [ ]:
from functools import partial
import rising.transforms as rtr
from rising.loading import DataLoader, default_transform_call
from rising.random import DiscreteParameter, UniformParameter

from kaggle_brain3d.utils import show_volume
from kaggle_brain3d.data import BrainScansDM, TRAIN_TRANSFORMS, VAL_TRANSFORMS
from kaggle_brain3d.transforms import RandomAffine, rising_zero_mean

# ==============================

dm = BrainScansDM(
    data_dir=PATH_DATASET,
    scan_types=["T2w"],
    vol_size=224,
    crop_thr=None,
    # crop_thr=1e-6,  # experimental crop threshold
    # batch_size=4,  # for full model training
    batch_size=2,  # for finetune head
    cache_dir=None,
    # in_memory=True,
    num_workers=2,
    train_transforms=rtr.Compose(TRAIN_TRANSFORMS, transform_call=default_transform_call),
    valid_transforms=rtr.Compose(VAL_TRANSFORMS, transform_call=default_transform_call),
)
dm.setup()
# dm.prepare_data(num_proc=3, dataset=dm.test_dataset)
print(f"Training batches: {len(dm.train_dataloader())}")
print(f"Validation batches: {len(dm.val_dataloader())}")
print(f"Test batches: {len(dm.test_dataloader())}")

# Quick view
for batch in dm.test_dataloader():
    for i in range(2):
        show_volume(batch["data"][i][0], fig_size=(6, 4), v_min_max=(-1., 3.))
    break

## Prepare 3D model

LightningModule is the core of PL, it wrappes all model related peaces, mainly:

- the model/architecture/weights
- evaluation metrics
- configs for optimizer and LR scheduler

In [ ]:
import torch
from torchsummary import summary
from kaggle_brain3d.models import LitBrainMRI, create_pretrained_medical_resnet
from monai.networks.nets import resnet10, resnet18, resnet34, resnet50, SEResNet50

# ==============================


PATH_CKPT = os.path.join(PATH_MODELS, "v10.ckpt")
net = resnet34(pretrained=False, spatial_dims=3, n_input_channels=1, num_classes=2)

model = LitBrainMRI(net=net)
# summary(model, input_size=(1, 128, 128, 128))
ckpt = torch.load(PATH_CKPT, map_location=torch.device('cpu'))
model.load_state_dict(ckpt['state_dict'])

## Predictions...

In [ ]:
model.eval()
model.cpu()
submission = []

for batch in dm.test_dataloader():
    print(batch.keys())
    print(batch.get("label"))
    imgs = batch.get("data")
    print(imgs.shape)
    with torch.no_grad():
        preds = model(imgs)
    print(preds)
    probs = torch.nn.functional.softmax(preds)
    print(probs)
    break

In [ ]:
from kaggle_brain3d.models import make_submission

df_submission = make_submission(model, dm.test_dataloader(), "cuda" if torch.cuda.is_available() else "cpu")
display(df_submission)
# df_submission.set_index("BraTS21ID", inplace=True)
df_submission[["MGMT_value"]].to_csv("submission.csv")

In [ ]:
df_submission[["MGMT_value"]].hist(bins=25)

In [ ]:
! cat submission.csv